In [18]:
#pip install selenium
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import os

In [19]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications": 2}
chrome_options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=chrome_options)

In [20]:
driver.get("https://www.cimb.com.my/en/personal/home.html")
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "svg[class='btn-overlay-close icon-bytesize absolute top-0 right-0 mt-3 mr-2 md:mt-6 md:mr-6 text-red-500 cursor-pointer z-100']"))).click()

In [21]:
main_category_elements = driver.find_elements(By.CSS_SELECTOR,'ul[class="accordion-content pl-0 list-none overflow-hidden h-0 md:h-auto h-0"]')

In [22]:
len(main_category_elements)

7

In [23]:
inner_htmls = [i.get_attribute('innerHTML') for i in main_category_elements]

In [24]:
inner_soups = [bs(inner_htmls[i], 'html.parser') for i in range(len(inner_htmls))]

In [25]:
main_categories = []
for i in range(len(inner_soups)):
    name = inner_soups[i].find_all('a')[0].get('href').split('/')[-2]
    main_categories.append(name)

In [26]:
main_categories

['digital-services',
 'accounts',
 'cards',
 'financing',
 'investments',
 'insurance-takaful',
 'rates-charges']

In [27]:
# first category 
data = {}
for i in range(len(main_categories)):
    category = main_categories[i]
    inner_soup = inner_soups[i]
    sub_cats = inner_soup.find_all('a')
    data[category] = {}
    for j in range(len(sub_cats)):
        sub_cat_link = sub_cats[j].get('href')
        sub_cat_name = sub_cat_link.split('/')[-1].split('.')[0]
        sub_cat_link_final = 'https://www.cimb.com.my'+sub_cat_link
        data[category][sub_cat_name] = sub_cat_link_final

In [28]:
import pandas as pd

In [33]:
data['digital-services']

{'cimb-clicks': 'https://www.cimb.com.my/en/personal/digital-services/cimb-clicks.html',
 'cimb-eva': 'https://www.cimb.com.my/en/personal/digital-services/cimb-eva.html',
 'cimb-partner-pay-solution': 'https://www.cimb.com.my/en/personal/digital-services/cimb-partner-pay-solution.html',
 'apply-for-products': 'https://www.cimb.com.my/en/personal/digital-services/apply-for-products.html',
 'cimb-qrpay': 'https://www.cimb.com.my/en/personal/digital-services/cimb-qrpay.html'}

In [32]:
test_url = data['digital-services']['cimb-clicks']
driver.get(test_url)

In [13]:
final_data = pd.DataFrame(columns = ['category','sub-category','product','product_link'])
final_rows = []
for cat in data.keys():
    for sub_cat in data[cat].keys():
        url = data[cat][sub_cat]
        driver.get(url)
        elements = driver.find_elements(By.CSS_SELECTOR,'a[class = "variant-bg-gradient w-full z-10"]')
        product_links = [i.get_attribute('href') for i in elements]
        print(cat,'|',sub_cat)
        print('-----------------------------------')
        if len(product_links)>0:
            for prod_link in product_links:
                product_name = prod_link.split('/')[-1].split('.')[0]
                row = (cat,sub_cat,product_name,prod_link)
        else:
            row = (cat,cat,sub_cat,url)
        
        final_rows.append(row)

digital-services | cimb-clicks
-----------------------------------
digital-services | cimb-eva
-----------------------------------
digital-services | cimb-partner-pay-solution
-----------------------------------
digital-services | apply-for-products
-----------------------------------
digital-services | cimb-qrpay
-----------------------------------
accounts | savings-account
-----------------------------------
accounts | current-account
-----------------------------------
accounts | fixed-term-deposit
-----------------------------------
cards | credit-card
-----------------------------------
cards | debit-card
-----------------------------------
cards | prepaid-card
-----------------------------------
cards | credit-card-services
-----------------------------------
financing | personal-financing
-----------------------------------
financing | property-financing
-----------------------------------
financing | auto-financing
-----------------------------------
investments | unit-trusts


In [14]:
for i in range(len(final_rows)):
    final_data.loc[i] = final_rows[i]

In [15]:
final_data.category.unique()

array(['digital-services', 'accounts', 'cards', 'financing',
       'investments', 'insurance-takaful', 'rates-charges'], dtype=object)

In [16]:
final_data

,category,sub-category,product,product_link
0,digital-services,digital-services,cimb-clicks,https://www.cimb.com.my/en/personal/digital-se...
1,digital-services,digital-services,cimb-eva,https://www.cimb.com.my/en/personal/digital-se...
2,digital-services,digital-services,cimb-partner-pay-solution,https://www.cimb.com.my/en/personal/digital-se...
3,digital-services,digital-services,apply-for-products,https://www.cimb.com.my/en/personal/digital-se...
4,digital-services,digital-services,cimb-qrpay,https://www.cimb.com.my/en/personal/digital-se...
5,accounts,savings-account,junior-savers-account-i,https://www.cimb.com.my/en/personal/day-to-day...
6,accounts,current-account,basic-current-account,https://www.cimb.com.my/en/personal/day-to-day...
7,accounts,fixed-term-deposit,golden-time-deposit-2-big-plus-for-50-plus,https://www.cimb.com.my/en/personal/day-to-day...
8,cards,credit-card,cimb-visa-infinite-credit-card,https://www.cimb.com.my/en/personal/day-to-day...
9,cards,debit-card,petronas-debit-mastercard,https://www.cimb.com.my/en/personal/day-to-day...


In [17]:
final_data.to_excel('cimb_product_details.xlsx',index = False)